## Imports

In [1]:
import random
import numpy as np
from string import punctuation

import nltk
from nltk.stem.snowball import SnowballStemmer as stemmer_fn

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import utils
from tensorflow.keras import layers

## Load data & configure NLTK

In [2]:
LANG = 'english'

stemmer = stemmer_fn(LANG)
nltk.download('punkt')

import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

[nltk_data] Downloading package punkt to /home/kuba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Extract corpus & intent from json

In [3]:
words = []
documents = []
classes = sorted(list(set([intent['tag'] for intent in intents['intents']])))
stop_words = set(list(punctuation))

def pre_process_words(wrds, stop):
    return [stemmer.stem(w.lower()) for w in wrds if w not in stop]

def pre_process_sentence(sentence, stop):
    wrds = nltk.word_tokenize(sentence)
    return [stemmer.stem(w.lower()) for w in wrds if w not in stop]

def bow_fn(sentence, words):
    wrds = pre_process_sentence(sentence, stop_words)
    bag = np.zeros((num_words))

    for i,w in enumerate(words):
        if w in wrds:
            bag[i] = 1
            
    return bag
            
# Go over the intents and their respective patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize patterns & skip stop words
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)

        # create pairs (tokenized sentence, intent)
        documents.append((wrds, intent['tag']))

'''
dictionary of words
- stemmed
- lowercase
- not in stop_words list
'''
words = pre_process_words(words, stop_words)
words = sorted(list(set(words)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

33 documents
8 classes ['More education', 'More experience', 'education', 'experience', 'goodbye', 'greeting', 'noanswer', 'thanks']
48 unique stemmed words ["'s", 'about', 'anyon', 'are', 'befor', 'bye', 'cheer', 'day', 'degre', 'did', 'do', 'educ', 'elabor', 'exact', 'experi', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'is', 'it', 'later', 'me', 'more', 'now', 'obtain', 'on', 'project', 'right', 'see', 'so', 'someth', 'somewher', 'studi', 'tell', 'thank', 'that', 'the', 'there', 'what', 'where', 'work', 'you', 'your']


## Create training data

In [35]:
num_documents = len(documents)
num_classes = len(classes)
num_words = len(words)
num_classes = len(classes)

X = np.zeros((num_documents, num_words))
y = np.zeros((num_documents, num_classes))

# training set, bag of words for each sentence
for j,doc in enumerate(documents):
    wrds, intent = doc
    wrds = pre_process_words(wrds, stop_words)
    
    for i,w in enumerate(words):
        if w in wrds:
            X[j,i] = 1

    y[j,classes.index(intent)] = 1

## Build & train simple model 

Here I actually don't mind the model overfitting the data

In [68]:
idx = np.arange(num_documents)
random.shuffle(idx)

X = X[idx]
y = y[idx]

num_neurons = 10

model = Sequential()
model.add(Dense(num_neurons, input_shape=(X.shape[1],)))
model.add(Dense(num_neurons))
model.add(Dense(num_neurons))
model.add(Dense(num_classes, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(np.array(X), np.array(y), epochs=500, batch_size=8)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 10)                490       
_________________________________________________________________
dense_21 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_22 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 88        
Total params: 798
Trainable params: 798
Non-trainable params: 0
_________________________________________________________________
Train on 33 samples
Epoch 1/500
33/33 [==============================] - 0s 13ms/sample - loss: 2.1530 - acc: 0.1515
Epoch 2/500
33/33 [==============================] - 0s 418us/sample - loss: 2.1205 - acc: 0.1818
Epoch 3/500
33/33 [====================

33/33 [==============================] - 0s 580us/sample - loss: 0.6691 - acc: 0.8788
Epoch 75/500
33/33 [==============================] - 0s 691us/sample - loss: 0.6607 - acc: 0.8485
Epoch 76/500
33/33 [==============================] - 0s 605us/sample - loss: 0.6530 - acc: 0.8485
Epoch 77/500
33/33 [==============================] - 0s 649us/sample - loss: 0.6412 - acc: 0.8485
Epoch 78/500
33/33 [==============================] - 0s 549us/sample - loss: 0.6269 - acc: 0.8485
Epoch 79/500
33/33 [==============================] - 0s 559us/sample - loss: 0.6103 - acc: 0.8485
Epoch 80/500
33/33 [==============================] - 0s 594us/sample - loss: 0.5902 - acc: 0.8788
Epoch 81/500
33/33 [==============================] - 0s 660us/sample - loss: 0.5785 - acc: 0.8788
Epoch 82/500
33/33 [==============================] - 0s 566us/sample - loss: 0.5635 - acc: 0.8788
Epoch 83/500
33/33 [==============================] - 0s 540us/sample - loss: 0.5503 - acc: 0.8788
Epoch 84/500
33/33 [===

Epoch 156/500
33/33 [==============================] - 0s 690us/sample - loss: 0.1709 - acc: 0.9394
Epoch 157/500
33/33 [==============================] - 0s 413us/sample - loss: 0.1697 - acc: 0.9394
Epoch 158/500
33/33 [==============================] - 0s 525us/sample - loss: 0.1676 - acc: 0.9394
Epoch 159/500
33/33 [==============================] - 0s 640us/sample - loss: 0.1658 - acc: 0.9394
Epoch 160/500
33/33 [==============================] - 0s 520us/sample - loss: 0.1640 - acc: 0.9394
Epoch 161/500
33/33 [==============================] - 0s 576us/sample - loss: 0.1624 - acc: 0.9394
Epoch 162/500
33/33 [==============================] - 0s 558us/sample - loss: 0.1613 - acc: 0.9394
Epoch 163/500
33/33 [==============================] - 0s 615us/sample - loss: 0.1598 - acc: 0.9394
Epoch 164/500
33/33 [==============================] - 0s 564us/sample - loss: 0.1599 - acc: 0.9394
Epoch 165/500
33/33 [==============================] - 0s 469us/sample - loss: 0.1567 - acc: 0.9394


33/33 [==============================] - 0s 581us/sample - loss: 0.1022 - acc: 0.9697
Epoch 238/500
33/33 [==============================] - 0s 753us/sample - loss: 0.0963 - acc: 0.9697
Epoch 239/500
33/33 [==============================] - 0s 673us/sample - loss: 0.0982 - acc: 0.9697
Epoch 240/500
33/33 [==============================] - 0s 775us/sample - loss: 0.0972 - acc: 0.9697
Epoch 241/500
33/33 [==============================] - 0s 562us/sample - loss: 0.0969 - acc: 0.9697
Epoch 242/500
33/33 [==============================] - 0s 848us/sample - loss: 0.0959 - acc: 0.9697
Epoch 243/500
33/33 [==============================] - 0s 1ms/sample - loss: 0.0948 - acc: 0.9697
Epoch 244/500
33/33 [==============================] - 0s 543us/sample - loss: 0.0941 - acc: 0.9697
Epoch 245/500
33/33 [==============================] - 0s 613us/sample - loss: 0.0934 - acc: 0.9697
Epoch 246/500
33/33 [==============================] - 0s 627us/sample - loss: 0.0932 - acc: 0.9697
Epoch 247/500
33

33/33 [==============================] - 0s 738us/sample - loss: 0.0664 - acc: 0.9697
Epoch 320/500
33/33 [==============================] - 0s 543us/sample - loss: 0.0726 - acc: 0.9697
Epoch 321/500
33/33 [==============================] - 0s 698us/sample - loss: 0.0721 - acc: 0.9697
Epoch 322/500
33/33 [==============================] - 0s 545us/sample - loss: 0.0748 - acc: 0.9697
Epoch 323/500
33/33 [==============================] - 0s 591us/sample - loss: 0.0761 - acc: 0.9697
Epoch 324/500
33/33 [==============================] - 0s 643us/sample - loss: 0.0747 - acc: 0.9697
Epoch 325/500
33/33 [==============================] - 0s 933us/sample - loss: 0.0730 - acc: 0.9697
Epoch 326/500
33/33 [==============================] - 0s 926us/sample - loss: 0.0707 - acc: 0.9697
Epoch 327/500
33/33 [==============================] - 0s 961us/sample - loss: 0.0685 - acc: 0.9697
Epoch 328/500
33/33 [==============================] - 0s 630us/sample - loss: 0.0655 - acc: 0.9697
Epoch 329/500


33/33 [==============================] - 0s 1ms/sample - loss: 0.0566 - acc: 0.9697
Epoch 402/500
33/33 [==============================] - 0s 829us/sample - loss: 0.0543 - acc: 0.9697
Epoch 403/500
33/33 [==============================] - 0s 952us/sample - loss: 0.0537 - acc: 0.9697
Epoch 404/500
33/33 [==============================] - 0s 766us/sample - loss: 0.0539 - acc: 0.9697
Epoch 405/500
33/33 [==============================] - 0s 746us/sample - loss: 0.0538 - acc: 0.9697
Epoch 406/500
33/33 [==============================] - 0s 891us/sample - loss: 0.0523 - acc: 0.9697
Epoch 407/500
33/33 [==============================] - 0s 896us/sample - loss: 0.0603 - acc: 0.9697
Epoch 408/500
33/33 [==============================] - 0s 732us/sample - loss: 0.0597 - acc: 0.9697
Epoch 409/500
33/33 [==============================] - 0s 694us/sample - loss: 0.0604 - acc: 0.9697
Epoch 410/500
33/33 [==============================] - 0s 697us/sample - loss: 0.0592 - acc: 0.9697
Epoch 411/500
33

33/33 [==============================] - 0s 794us/sample - loss: 0.0484 - acc: 0.9697
Epoch 484/500
33/33 [==============================] - 0s 704us/sample - loss: 0.0483 - acc: 0.9697
Epoch 485/500
33/33 [==============================] - 0s 732us/sample - loss: 0.0449 - acc: 0.9697
Epoch 486/500
33/33 [==============================] - 0s 656us/sample - loss: 0.0498 - acc: 0.9697
Epoch 487/500
33/33 [==============================] - 0s 699us/sample - loss: 0.0546 - acc: 0.9697
Epoch 488/500
33/33 [==============================] - 0s 581us/sample - loss: 0.0561 - acc: 0.9697
Epoch 489/500
33/33 [==============================] - 0s 837us/sample - loss: 0.0555 - acc: 0.9697
Epoch 490/500
33/33 [==============================] - 0s 749us/sample - loss: 0.0546 - acc: 0.9697
Epoch 491/500
33/33 [==============================] - 0s 726us/sample - loss: 0.0533 - acc: 0.9697
Epoch 492/500
33/33 [==============================] - 0s 488us/sample - loss: 0.0507 - acc: 0.9697
Epoch 493/500


In [69]:
context = {}

def inference(sentence, threshold, show_details=False):
    p = bow_fn(sentence, words)
    p = np.expand_dims(p,axis=0)

    results = model.predict(p)[0]
    y_pred = np.argmax(results)
    
    if show_details:
        print(results, y_pred)
    
    if results[y_pred] > threshold:
        return y_pred
    else:
        return None

In [70]:
text = 'Talk to you later jakub'

threshold = 0.3
t = inference(text, threshold)
print(classes[t])

goodbye


In [71]:
def response(sentence, userID='user_ID', show_details=False):
    results = inference(sentence, threshold, show_details)

    if results is not None:
        intent_pred = classes[results]

        for intent in intents['intents']:
            if intent['tag'] == intent_pred:
                if 'context_set' in intent:
                    context[userID] = intent['context_set']

                    if show_details: 
                        print ('context:', intent['context_set'])

                # check if this intent is contextual and applies to this user's conversation
                if not 'context_filter' in intent or \
                    (userID in context and 'context_filter' in intent and intent['context_filter'] == context[userID]):
                    if show_details: 
                        print('tag:', intent['tag'])

                    return print(random.choice(intent['responses']))

In [72]:
print(classes)

context = {}
print('\n\n',context)
response("What's your education", userID='123', show_details=True)
print('\n\n',context)
response("tell me more about it", userID='123', show_details=True)
response('alright thanks', userID='123', show_details=True)

['More education', 'More experience', 'education', 'experience', 'goodbye', 'greeting', 'noanswer', 'thanks']


 {}
[2.1008150e-06 1.6923414e-08 9.9464875e-01 3.8202158e-03 5.4121990e-04
 5.1938791e-09 1.0431967e-07 9.8760403e-04] 2
context: more_education
tag: education
I recently graduated from master degree.


 {'123': 'more_education'}
[4.9114689e-01 5.0858384e-01 1.0470107e-05 4.3474390e-07 9.5000200e-05
 1.2722837e-04 3.3919070e-05 2.2319523e-06] 1
[7.2793860e-05 8.2190958e-08 1.1453610e-06 1.8612598e-08 1.1317981e-04
 5.7426109e-06 3.6355166e-03 9.9617153e-01] 7
context: 
tag: thanks
My pleasure


In [73]:
context = {}
response("What's your education", userID='12345')
response("tell me more", userID='12345')
response("tell me more", userID='12345')
response('alright thanks', userID='12345')
response("cheers")
response("bye")

print('\n\n')
response("Where do you work", userID='15')
response("tell me more", userID='15')
response("tell me more", userID='15')
response('alright thanks', userID='15')
response("cheers")
response("bye")

I recently graduated from master degree.
You're welcome!
You're welcome!
See you later, thanks for visiting



I did some projects within CV and NLP
My career started as web developer, but I always wanted to work with ML.
Sample projects are listed on my portfolio website: https://jakubczerny.wixsite.com/portfolio
You're welcome!
My pleasure
Bye! Come back again soon.
